## Part 3 Decision Tree

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_score
import scipy

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

folder_dir = './cifar-10-batches-py/' 

load cifar data

In [3]:

import numpy as np
folder_dir = './cifar-10-batches-py/' 
metadata_dict = unpickle(folder_dir+"batches.meta")
label_names = np.array([label.decode('ascii') for label in metadata_dict[b'label_names']])

cifar_train_data = None
cifar_train_filenames = []
cifar_train_labels = []

for i in range(1,6):
    cifar_train_data_dict = unpickle(folder_dir + "data_batch_" + str(i))
    if i == 1:
        cifar_train_data = cifar_train_data_dict[b'data']
    else:
        cifar_train_data = np.vstack((cifar_train_data, cifar_train_data_dict[b'data']))
    cifar_train_labels += cifar_train_data_dict[b'labels']

cifar_train_data = cifar_train_data.reshape((len(cifar_train_data), 3, 32, 32))
cifar_train_data = np.rollaxis(cifar_train_data, 1, 4) # only if plt.imshow()

cifar_test_data_dict = unpickle(folder_dir + "test_batch")
cifar_test_data = cifar_test_data_dict[b'data']
cifar_test_filenames = cifar_test_data_dict[b'filenames']
cifar_test_labels = cifar_test_data_dict[b'labels']

cifar_test_data = cifar_test_data.reshape((len(cifar_test_data), 3, 32, 32))
# cifar_test_data = np.rollaxis(cifar_test_dsata, 1, 4)


cifar_train_data = cifar_train_data.reshape(50000, -1)
cifar_test_data = cifar_test_data.reshape(10000, -1)
print(cifar_train_data.shape)
print(cifar_test_data.shape)

(50000, 3072)
(10000, 3072)


rescaling from [0,255] to [-1,1]

In [4]:
X_train = cifar_train_data/127.5-1
X_test = cifar_test_data/127.5-1
y_train = np.array(cifar_train_labels)
y_test = np.array(cifar_test_labels)
print("X_train.shape", X_train.shape)
print("X_test.shape", X_test.shape)
print("y_test.shape", y_test.shape)
print("y_train.shape", y_train.shape)

X_train.shape (50000, 3072)
X_test.shape (10000, 3072)
y_test.shape (10000,)
y_train.shape (50000,)


Instantiate the Decision Tree model

In [5]:
tree = sklearn.tree.DecisionTreeClassifier()

Train the DecisionTree model

In [8]:
param_distribution = {'max_depth': range(1,75), 'min_samples_split': range(2,50)}

best_tree = sklearn.model_selection.RandomizedSearchCV(tree, param_distribution, n_iter=200, cv=2, random_state=0, verbose=3).fit(X_train, y_train)
    
print('best Hyperparameters = ' + str(best_tree.best_params_))


Fitting 2 folds for each of 200 candidates, totalling 400 fits
[CV 1/2] END max_depth=21, min_samples_split=31;, score=0.272 total time= 1.2min
[CV 2/2] END max_depth=21, min_samples_split=31;, score=0.270 total time= 1.2min
[CV 1/2] END max_depth=10, min_samples_split=33;, score=0.284 total time=  53.3s
[CV 2/2] END max_depth=10, min_samples_split=33;, score=0.285 total time=  47.7s
[CV 1/2] END max_depth=20, min_samples_split=48;, score=0.275 total time= 1.0min
[CV 2/2] END max_depth=20, min_samples_split=48;, score=0.279 total time= 1.0min
[CV 1/2] END ..max_depth=5, min_samples_split=6;, score=0.255 total time=  21.5s
[CV 2/2] END ..max_depth=5, min_samples_split=6;, score=0.256 total time=  21.8s
[CV 1/2] END max_depth=18, min_samples_split=11;, score=0.258 total time= 1.1min
[CV 2/2] END max_depth=18, min_samples_split=11;, score=0.261 total time= 1.1min
[CV 1/2] END max_depth=36, min_samples_split=48;, score=0.275 total time=  59.6s
[CV 2/2] END max_depth=36, min_samples_split=4

In [7]:
import pickle
Part_3_Decision_tree = {'model':best_tree}

with open('Part_3_Decision_tree.dictionary', 'wb') as config_dictionary_file:
    pickle.dump(Part_3_Decision_tree, config_dictionary_file)

Testing the DecisionTree model

In [ ]:
best_tree = sklearn.tree.DecisionTreeClassifier(max_depth=10, min_samples_split=49).fit(X_train, y_train)
tree_score = best_tree.score(X_test, y_test)
print("score", tree_score)

Plotting the DecisionTree

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

plt.figure(figsize=(21,21))
plot_tree(best_tree, max_depth = 2, fontsize=14);